학습은 로컬로 진행하였습니다.
아래의 코드는 완전하지는 않고 흐름 정도만 나타나있습니다.

1. label의 데이터를 yolov5가 사용할 수 있는 형태의 txt파일로 만들어준다. ({레이블} {x_center} {y_center} {width} {height} 형태)

2. 이미지를 train과 validation으로 나누어 그 정보를 txt파일에 저장한다.

3. 레이블과 이미지셋 정보를 담은 yaml파일을 생성한다.

4. yaml파일과 yolov5s모델을 통해 학습시킨다.

In [ ]:
from glob import glob
from sklearn.model_selection import train_test_split
import os
import json

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [12]:
# 로컬로 진행하였음
%cd C:/Users/vscha/Section3/AI_16_CP1_DS/

C:\Users\vscha\Section3\AI_16_CP1_DS


In [ ]:
# Label 데이터의 json을 txt로 변환
json_files = [f for f in os.listdir('.\data\json') if f.endswith('.json')]


for json_file in json_files:
    with open(os.path.join('.\data\json', json_file), encoding='utf-8') as f:
        data = json.load(f)
        items = data['FILE'][0]['ITEMS']
        img_width, img_height = map(
            int, data['FILE'][0]['RESOLUTION'].split("*"))

        classes = {'정상차량': 0, '불법차량': 1}

        for item in items:
            class_name = item["PACKAGE"]
            class_id = classes[class_name]

            box = item['BOX'].split(',')
            x1, y1, x2, y2 = map(float, box)

            dw, dh = 1. / img_width, 1. / img_height

            x, y, w, h = x1 + x2 / 2.0, y1 + y2 / 2.0, x2, y2

            x_center, width = x * dw, w * dw
            y_center, height = y * dh, h * dh

            yolo_label = f"{class_id} {x_center} {y_center} {width} {height}"

            filename = os.path.splitext(json_file)[0] + '.txt'
            with open(os.path.join('.\dataset\labels', filename), 'w') as f:
                f.write(yolo_label)

In [13]:
# 이미지 리스트를 받아오기
def get_image_list(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.jpg')]

# 이미지 리스트를 test셋과 val셋으로 나누기
def split_data(train_size=0.8):
    img_list = get_image_list('./dataset/images')
    train_img_list, val_img_list = train_test_split(img_list, test_size=1-train_size)
    with open('./dataset/train.txt', 'w') as f:
        f.write('\n'.join(train_img_list) + '\n')
    with open('./dataset/val.txt', 'w') as f:
        f.write('\n'.join(val_img_list) + '\n')

split_data()

'.\\dataset\\images\\A01_B02_C00_D01_0818_E05_F06_159_5.jpg\n'

In [ ]:
# 레이블과 데이터에 맞는 yaml파일을 만든 뒤 학습을 진행

!python ./models/yolov5/train.py --img 640 --batch 16 --epochs 50 --data ./dataset/data.yaml --cfg ./models/yolov5/models/yolov5s.yaml --weights yolov5.pt --name final_truck_yolov5s_results

In [18]:
from IPython.display import Image
import os

# 학습 결과 확인
val_img_path = val_img_list[5]

!python ./models/yolov5/detect.py --weights ./models/yolov5/runs/train/final_truck_yolov5s_results/weights/best.pt --img 640 --conf 0.5 --source "{val_img_path}"

detect: weights=['./models/yolov5/runs/train/final_truck_yolov5s_results/weights/best.pt'], source=.\dataset\images\A01_B02_C00_D01_0818_E05_F06_344_5.jpg, data=models\yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=models\yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-120-g3e55763 Python-3.8.16 torch-1.13.1+cpu CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 C:\Users\vscha\Section3\AI_16_CP1_DS\dataset\images\A01_B02_C00_D01_0818_E05_F06_344_5.jpg: 384x640 1 legal, 123.0ms
Speed: 1.0ms pre-process, 123.0ms inference, 1.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to models\yolov5\runs\detect\exp4
